In [1]:
# SWAMI KARUPPASWAMI THUNNAI

In [3]:
# Import all the necessary libararies
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from nltk.tokenize import TweetTokenizer
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# Get the data-set
data_set = pd.read_csv("processed.csv")

In [5]:
# This is our independent variable - The tweet data
X = data_set.iloc[:, :-1]

In [6]:
# This is our dependent variable - Positive Negative
y = data_set.iloc[:, -1]

In [7]:
# Convert the Categroical data (negative and positive) to numberical data 0 and 1
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [8]:
corpus = []

In [9]:
def tweet_preprocessor(tweet_text):
    """
    Will preprocess the tweets like remove the usernames, hashtags, urls and
    will preprocess the content suitable for NLP.
    :return: preprocessed tweet
    """
    # convert the tweet to the lower case
    tweet_text = tweet_text.lower()
    tokenizer = TweetTokenizer()
    words = tokenizer.tokenize(tweet_text)
    preprocessed_words = []
    for word in words:
        if word.startswith("@") or word.startswith("#") or word.startswith("https://") or word.startswith("http://"):
            pass
        else:
            preprocessed_words.append(word)
    # remove the stop words
    stopwords_removed = [word for word in preprocessed_words if not word in stopwords.words("english")]
    # make sure they are words
    punc_remover = lambda word: re.sub("[^A-Za-z]", " ", word)
    # get the pure words without punctuation
    pure_words = list(map(punc_remover, stopwords_removed))
    # Stem the words
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in pure_words]
    # strip the words
    stripper = lambda word: word.strip()
    stemmed_words = list(map(stripper, stemmed_words))
    stemmed_words = filter(None, stemmed_words)
    # Get our processed tweet
    processed_tweet = " ".join(stemmed_words)
    return processed_tweet
    

In [10]:
result = list(X["Tweet"])

In [11]:
for i in result:
    value = tweet_preprocessor(i)
    corpus.append(value)
print("Completed")

Completed


In [12]:
cv = CountVectorizer(max_features=1500)

In [13]:
# All the text data is converted into bag of words model
X = cv.fit_transform(corpus).toarray()

In [14]:
# Split 80% to traning and 20% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
SVMClassifier=SVC()

In [16]:
SVMClassifier.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
Y_pred = SVMClassifier.predict(X_test)

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.4870360955770208

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[ 955,    0],
       [1009,    3]])